In [125]:
import cv2
import numpy as np
import os

def view(img): # показ изображения
    img = cv2.resize(img, (502, 628), cv2.INTER_NEAREST)
    cv2.imshow('LEAFS', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [141]:
# Создаем контуры листьев
img1 = cv2.imread('p2.png')

low = np.array([200, 200, 200])
high = np.array([255, 255, 255])
mask = cv2.inRange(img1, low, high)
img1[mask > 0] = ([255, 255, 255])

gray = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY) 
gray = cv2.medianBlur(gray, 69)
_, thr = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
contours, hierarchy = cv2.findContours(thr, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
hierarchy = hierarchy[0][1:]
contours = contours[1:]
max_area = max([cv2.contourArea(x, False) for x in contours]) 
contours = [cv2.convexHull(contours[i], False) for i in range(len(contours)) if cv2.contourArea(contours[i]) > max_area / 3]
#cv2.drawContours(img1, contours, -1, (0, 0, 250), 10)

if 'storage' not in os.listdir():
    os.makedirs('storage')
    
for idx in range(len(contours)):
    mask = np.zeros_like(img1) # вектор с нулями, каждый пиксель представлен 0 - черный цвет
    cv2.drawContours(mask, contours, idx, [255, 255, 255], -1) # Рисуем на mask контур из contours под индексом idx белого цвета (-1 - заполненный контур)
    # Получаем черное изображение с белым пятном листа
    out = np.zeros_like(img) # Создаем еще один нулевой вектор, в него будем записывать изображение
    out[mask == [255, 255, 255]] = img1[mask == [255, 255, 255]] # пиксели out[аналогичные пиксели в mask = 255] заменяются на пиксели img[аналогичные пиксели в mask = 255]
    
    x, y, w, h = cv2.boundingRect(contours[idx])
    y = y - 100
    x = x - 100
    ROI = out[y:y+h+200, x:x+w+200]
    
    low = np.array([0, 0, 0])
    high = np.array([1, 1, 1])
    mask = cv2.inRange(ROI, low, high)
    ROI[mask > 0] = ([255, 255, 255])
    cv2.imwrite(f'storage/{idx+1} list.png', ROI)

In [122]:
thresh = cv2.threshold(out, [0, 0 ,0] , [255, 255, 255], cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]
viewImage(thresh)
# Find contour and sort by contour area
cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

# Find bounding box and extract ROI
for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    y = y - 100
    x = x - 100
    ROI = out[y:y+h+200, x:x+w+200]
    break

viewImage(ROI)


error: OpenCV(4.8.1) :-1: error: (-5:Bad argument) in function 'threshold'
> Overload resolution failed:
>  - Argument 'thresh' can not be treated as a double
>  - Argument 'thresh' can not be treated as a double


In [5]:
import numpy as np
import cv2
img = cv2.imread('p2.png') # Read in your image
viewImage(img)
contours = leafs_contours
idx = 0 # Индекс контура
mask = np.zeros_like(img) # вектор с нулями, каждый пиксель представлен 0 - черный цвет

cv2.drawContours(mask, contours, idx, [255, 255, 255], -1) # Рисуем на mask контур из contours под индексом idx белого цвета (-1 - заполненный контур)
# Получаем черное изображение с белым пятном листа
viewImage(mask)
out = np.zeros_like(img) # Создаем еще один нулевой вектор, в него будем записывать изображение

out[mask == [255, 255, 255]] = img_old[mask == [255, 255, 255]] # пиксели out[аналогичные пиксели в mask = 255] заменяются на пиксели img[аналогичные пиксели в mask = 255]

viewImage(out)

thresh = cv2.threshold(out, [0, 0 ,0] , [255, 255, 255], cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]
viewImage(thresh)
# Find contour and sort by contour area
cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

# Find bounding box and extract ROI
for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    y = y - 100
    x = x - 100
    ROI = out[y:y+h+200, x:x+w+200]
    break

viewImage(ROI)
